In [1]:
import mysql.connector
import config
import pandas as pd
import numpy as np
import random
import time

start = time.time()

PATH = config.PATH
db = mysql.connector.connect(
    host=config.host,
    user=config.user,
    passwd=config.passwd,
    database=config.database
)
cursor = db.cursor()

In [2]:
def rand_choice(list):
    return list[random.randrange(0, len(list))]

def rand_date(start, end, format, prop):
    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(format, time.localtime(ptime))


# Uploading users
## We have | User ID | First Name | Last Name | Gender |  Home Town | City |
## We don't have | Relationship Status |



In [3]:
## 00 single, 01 Engaged, 10 Married, 11 Complicated
relationship_types = [int('00', base = 2), int('01', base = 2), int('10', base = 2), int('11', base = 2)]

df = pd.read_csv(PATH + 'facebook_users_5000_max.csv')

print(df.head())
for index, row in df.iterrows():
    # Only male or female in the dataset, but using 2 BITS so 10 = unspecified
    gender = int('00', base = 2) if row['Gender'] == 'Male' else int('01', base = 2)
    #Random choice for a relationship status
    relationship_status = rand_choice(relationship_types)
    sql = "INSERT INTO Users (userID, firstName, lastName, gender, homeTown, currentCity, relationshipStatus) VALUES ({0}, \"{1}\", \"{2}\", {3}, \"{4}\", \"{5}\", {6});".format(
    row['UserID'], row['FirstName'], row['LastName'], gender, row['Hometown'], row['City'], relationship_status)
    cursor.execute(sql)
    #print(sql)
    db.commit()

   UserID FirstName   LastName  Gender          Hometown            City
0       1   Phillip  Alldridge    Male   North Amayatown     Jermeymouth
1       2     Logan     Norris  Female       Reichelland    East Daphnee
2       3     Holly      Grant  Female       Wuckertberg       Jamirfurt
3       4  Chadwick    Gardner    Male  New Justinaville     West Manley
4       5      Bart     Fenton    Male     South Zackery  New Dimitriton


# Uploading Workplaces
## We have the name and the ID is auto incrementing

In [4]:
df = pd.read_csv(PATH + 'workplace.csv')

# Key value list {work place name : work place id}
workplaces_dict = {}
num_workplaces = 0
print(df.head())
for index, row in df.iterrows():
    if row['Workplace'] not in workplaces_dict and not pd.isna(row['Workplace']):
        num_workplaces += 1
        workplaces_dict.update({row['Workplace'] : num_workplaces})
        sql = "INSERT INTO Workplaces(workplaceName) VALUES(\"{0}\");".format(row['Workplace'])
        cursor.execute(sql)
        db.commit()
        #print(sql)

   UserID         Workplace
0       1             AECOM
1       2               NaN
2       3  21st Century Fox
3       3        Apple Inc.
4       3  Metro Cash&Carry


# Uploading Worker
## We have | User ID | Workplace ID |
## We don't have | Start date | End date | 

In [5]:
df = pd.read_csv(PATH + 'workplace.csv')
df['date_started'] = np.empty((len(df), 0)).tolist()
df['date_left'] = np.empty((len(df), 0)).tolist()
date_format = '%Y-%m-%d'

# Drop any user over 5,000 - Data file has been made smaller to lower load on uni servers
workerIndex = df[df['UserID'] > 5000].index
df.drop(workerIndex , inplace=True)

print(df.head())
print(df.tail())

for index, row in df.iterrows():
    if not pd.isna(row['Workplace']):
        if row['UserID'] == df.iloc[index - 1, 0]:
            df.iloc[index, 2] = rand_date(df.iloc[index - 1, 3], "2020-1-1", date_format, random.random())
        else:
            df.iloc[index, 2] = rand_date("1980-1-1", "2020-1-1", date_format, random.random())
        if row['UserID'] == df.iloc[index + 1, 0]:
            df.iloc[index, 3] = rand_date(df.iloc[index, 2], "2020-1-1", date_format, random.random())
        #Fetch the updated row
        curr_row = df.iloc[index, :]
        sql = ""
        if curr_row['date_left'] == []:
            sql = "INSERT INTO Workers (userID, workplaceID, dateStarted) VALUES({0}, {1}, \"{2}\");".format(
            curr_row['UserID'], workplaces_dict[curr_row['Workplace']], curr_row['date_started'])
        else:
            sql = "INSERT INTO Workers (userID, workplaceID, dateStarted, dateLeft) VALUES({0}, {1}, \"{2}\", \"{3}\");".format(
            curr_row['UserID'], workplaces_dict[curr_row['Workplace']], curr_row['date_started'], curr_row['date_left'])
        cursor.execute(sql)
        #print(sql)
        db.commit()
    
        

   UserID         Workplace date_started date_left
0       1             AECOM           []        []
1       2               NaN           []        []
2       3  21st Century Fox           []        []
3       3        Apple Inc.           []        []
4       3  Metro Cash&Carry           []        []
      UserID   Workplace date_started date_left
6893    4996  ExxonMobil           []        []
6894    4997         NaN           []        []
6895    4998         NaN           []        []
6896    4999         NaN           []        []
6897    5000         NaN           []        []


# Uploading Universities
## We have the name and the ID is auto incrementing
## Pretty much a copy and paste of Workplaces

In [6]:
df = pd.read_csv(PATH + 'universities.csv')

universities_dict = {}
num_unis = 0
print(df.head())

for index, row in df.iterrows():
    if row['University'] not in universities_dict and not pd.isna(row['University']):
        num_unis += 1
        universities_dict.update({row['University'] : num_unis})
        sql = "INSERT INTO Universities(universityName) VALUES(\"{0}\");".format(row['University'])
        #print(sql)
        cursor.execute(sql)
        db.commit()

   UserID                     University
0       1                            NaN
1       2         University of Oklahoma
2       2         University of Virginia
3       3  University of Texas at Austin
4       4     University at Buffalo SUNY


# Uploading Students
## We have | User ID | universityName Name |
## We don't have | Start date | End date | 
## Pretty much a full copy of Workers

In [7]:
df = pd.read_csv(PATH + 'universities.csv')
df['date_started'] = np.empty((len(df), 0)).tolist()
df['date_left'] = np.empty((len(df), 0)).tolist()
date_format = '%Y-%m-%d'

# Drop any user over 5,000 - Data file has been made smaller to lower load on uni servers
studentIndex = df[df['UserID'] > 5000].index
df.drop(studentIndex , inplace=True)

print(df.head())
print(df.tail())

for index, row in df.iterrows():
    if not pd.isna(row['University']):
        if row['UserID'] == df.iloc[index - 1, 0]:
            df.iloc[index, 2] = rand_date(df.iloc[index - 1, 3], "2020-1-1", date_format, random.random())
        else:
            df.iloc[index, 2] = rand_date("1980-1-1", "2020-1-1", date_format, random.random())
        if row['UserID'] == df.iloc[index + 1, 0]:
            df.iloc[index, 3] = rand_date(df.iloc[index, 2], "2020-1-1", date_format, random.random())
        #Fetch the updated row
        curr_row = df.iloc[index, :]
        sql = ""
        if curr_row['date_left'] == []:
            sql = "INSERT INTO Students (userID, uniID, dateStarted) VALUES({0}, {1}, \"{2}\");".format(
            curr_row['UserID'], universities_dict[curr_row['University']], curr_row['date_started'])
        else:
            sql = "INSERT INTO Students (userID, uniID, dateStarted, dateLeft) VALUES({0}, {1}, \"{2}\", \"{3}\");".format(
            curr_row['UserID'], universities_dict[curr_row['University']], curr_row['date_started'], curr_row['date_left'])
        #print(sql)
        cursor.execute(sql)
        db.commit()
        

   UserID                     University date_started date_left
0       1                            NaN           []        []
1       2         University of Oklahoma           []        []
2       2         University of Virginia           []        []
3       3  University of Texas at Austin           []        []
4       4     University at Buffalo SUNY           []        []
      UserID                University date_started date_left
6863    4996                       NaN           []        []
6864    4997                       NaN           []        []
6865    4998  University of Pittsburgh           []        []
6866    4999                       NaN           []        []
6867    5000                       NaN           []        []


# Uploading Friendships
## No data to create here!

In [8]:
df = pd.read_csv(PATH + 'facebook-friendships-5000_max.csv')

print(df.head())

for index, row in df.iterrows():
    sql = "INSERT INTO Friendships(userID_1, userID_2) VALUES ({0}, {1});".format(
    row['UserID_1'], row['UserID_2'])
    #print(sql)
    cursor.execute(sql)
    db.commit()
    

   UserID_1  UserID_2
0         1         2
1         1         3
2         1         4
3         1         5
4         1         6


# Uploading Messages
## No data to create here!

In [9]:
df = pd.read_csv(PATH + 'messages.csv')

print(df.head())

indexMessages = df[df['UserID_1'] > 5000].index
df.drop(indexMessages , inplace=True)

indexMessages = df[df['UserID_2'] > 5000].index
df.drop(indexMessages , inplace=True)

for index, row in df.iterrows():
    sql = "INSERT INTO Messages(senderID, recipientID, sentTime, messageText) VALUES({0}, {1}, \"{2}\", \"{3}\");".format(
    row['UserID_1'], row['UserID_2'], row['Date_Time'], row['Message'])
    cursor.execute(sql)
    #print(sql)
    db.commit()

   UserID_1  UserID_2            Date_Time  \
0         1         2  2018-12-27 11:36:41   
1         1         5  2018-10-12 03:12:01   
2         1         9  2018-10-18 23:26:43   
3         1        16  2018-12-29 08:33:17   
4         1        20  2019-01-04 16:11:44   

                                             Message  
0  risus semper porta volutpat quam pede lobortis...  
1  lectus pellentesque eget nunc donec quis orci ...  
2  iaculis diam erat fermentum justo nec condimen...  
3                                      morbi a ipsum  
4  erat vestibulum sed magna at nunc commodo plac...  


## Done uploading the data!
## How long it took:

In [10]:
end = time.time()
print(end - start)

7260.322409152985
